El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Preparación de librerias


<div class="alert alert-block alert-info">
<b>pip install tabulate</b> <a class=“tocSkip”></a>
</div>

In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor

import time
import tempfile
from tabulate import tabulate


## Preparación de datos

In [2]:
# Cargar los datos
try:
    # no windows
    data = pd.read_csv('/datasets/car_data.csv')
    print("Archivo cargado exitosamente desde '/datasets/car_data.csv'")
except FileNotFoundError:
    try:
        # windows
        data = pd.read_csv('datasets/car_data.csv')
        print("Archivo cargado exitosamente desde 'datasets/car_data.csv'")
    except FileNotFoundError:
        print("Error: el archivo 'car_data.csv' no se encuentra en ninguna de las rutas especificadas.")

Archivo cargado exitosamente desde 'datasets/car_data.csv'


## Revisión de datos

In [3]:
# Mostrar las primeras filas del dataset
print("Primeras filas del dataset:")
display(data.head())

# Obtener información sobre el dataset
print("Información del dataset:")
display(data.info())

# Describir el dataset para obtener estadísticas básicas
print("Descripción del dataset:")
display(data.describe(include='all'))

# Verificar las columnas presentes en el DataFrame
print("Columnas en el DataFrame:", data.columns)


Primeras filas del dataset:


,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17


Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object


None

Descripción del dataset:


,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
count,354369,354369.000000,316879,354369.000000,334536,354369.000000,334664,354369.000000,354369.000000,321474,354369,283215,354369,354369.0,354369.000000,354369
unique,15470,NaN,8,NaN,2,NaN,250,NaN,NaN,7,40,2,109,NaN,NaN,18592
top,05/03/2016 14:25,NaN,sedan,NaN,manual,NaN,golf,NaN,NaN,petrol,volkswagen,no,03/04/2016 00:00,NaN,NaN,07/04/2016 07:16
freq,66,NaN,91457,NaN,268251,NaN,29232,NaN,NaN,216352,77013,247161,13719,NaN,NaN,654
mean,NaN,4416.656776,NaN,2004.234448,NaN,110.094337,NaN,128211.172535,5.714645,NaN,NaN,NaN,NaN,0.0,50508.689087,NaN
std,NaN,4514.158514,NaN,90.227958,NaN,189.850405,NaN,37905.341530,3.726421,NaN,NaN,NaN,NaN,0.0,25783.096248,NaN
min,NaN,0.000000,NaN,1000.000000,NaN,0.000000,NaN,5000.000000,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,1050.000000,NaN,1999.000000,NaN,69.000000,NaN,125000.000000,3.000000,NaN,NaN,NaN,NaN,0.0,30165.000000,NaN
50%,NaN,2700.000000,NaN,2003.000000,NaN,105.000000,NaN,150000.000000,6.000000,NaN,NaN,NaN,NaN,0.0,49413.000000,NaN
75%,NaN,6400.000000,NaN,2008.000000,NaN,143.000000,NaN,150000.000000,9.000000,NaN,NaN,NaN,NaN,0.0,71083.000000,NaN


Columnas en el DataFrame: Index(['DateCrawled', 'Price', 'VehicleType', 'RegistrationYear', 'Gearbox',
       'Power', 'Model', 'Mileage', 'RegistrationMonth', 'FuelType', 'Brand',
       'NotRepaired', 'DateCreated', 'NumberOfPictures', 'PostalCode',
       'LastSeen'],
      dtype='object')


## Preprocesamiento de Datos 

In [4]:
# Verificar si todas las columnas categóricas están presentes en el DataFrame
categorical_cols = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']
missing_cols = [col for col in categorical_cols if col not in data.columns]
if missing_cols:
    print(f"Advertencia: Las siguientes columnas categóricas están faltando en el DataFrame y serán excluidas: {missing_cols}")
    categorical_cols = [col for col in categorical_cols if col in data.columns]

# Eliminamos las columnas innecesarias
data = data.drop(['DateCrawled', 'DateCreated', 'LastSeen', 'NumberOfPictures'], axis=1)

# Manejo de valores faltantes
data = data.dropna()

# Separar las características y la variable objetivo
X = data.drop('Price', axis=1)
y = data['Price']

# Verificar las columnas después de manejar los valores faltantes
print("Columnas en X después de manejar valores faltantes:", X.columns)

Columnas en X después de manejar valores faltantes: Index(['VehicleType', 'RegistrationYear', 'Gearbox', 'Power', 'Model',
       'Mileage', 'RegistrationMonth', 'FuelType', 'Brand', 'NotRepaired',
       'PostalCode'],
      dtype='object')


## One-Hot Encoding

In [5]:
# One-Hot Encoding para los modelos que lo necesitan
encoder = OneHotEncoder(drop='first', sparse_output=False)  # Actualizado
if categorical_cols:
    X_encoded = encoder.fit_transform(X[categorical_cols])
    encoded_cols = encoder.get_feature_names_out(categorical_cols)
    encoded_df = pd.DataFrame(X_encoded, columns=encoded_cols)
    
    # Reemplazar las columnas categóricas con las codificadas
    X = X.drop(categorical_cols, axis=1).reset_index(drop=True)
    X = pd.concat([X, encoded_df], axis=1)

## Dividir los Datos en Conjuntos de Entrenamiento y Prueba

In [6]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Entrenamiento del modelo 

### Regresión Lineal

In [7]:
import sklearn
sklearn.__version__

'1.5.0'

In [8]:

# Entrenar el modelo
lr = LinearRegression()
lr.fit(X_train, y_train)

# Hacer predicciones
y_pred_lr = lr.predict(X_test)

# Calcular el RMSE
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))
print(f'RMSE Regresión Lineal: {rmse_lr}')


RMSE Regresión Lineal: 2701.260670522398


### Árbol de Decisión

In [9]:
# Entrenar el modelo
dt = DecisionTreeRegressor(random_state=42)
dt.fit(X_train, y_train)

# Hacer predicciones
y_pred_dt = dt.predict(X_test)

# Calcular el RMSE
rmse_dt = np.sqrt(mean_squared_error(y_test, y_pred_dt))
print(f'RMSE Árbol de Decisión: {rmse_dt}')
 

RMSE Árbol de Decisión: 2098.711200373837


### Bosque Aleatorio

In [10]:
# Entrenar el modelo
rf = RandomForestRegressor(random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

# Hacer predicciones
y_pred_rf = rf.predict(X_test)

# Calcular el RMSE
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
print(f'RMSE Bosque Aleatorio: {rmse_rf}')
 

RMSE Bosque Aleatorio: 1583.94459792293


### LightGBM

In [11]:
# Entrenar el modelo
lgb_model = lgb.LGBMRegressor(random_state=42, n_jobs=-1)
lgb_model.fit(X_train, y_train,)

# Hacer predicciones
y_pred_lgb = lgb_model.predict(X_test)

# Calcular el RMSE
rmse_lgb = np.sqrt(mean_squared_error(y_test, y_pred_lgb))
print(f'RMSE LightGBM: {rmse_lgb}')

 

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002528 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1176
[LightGBM] [Info] Number of data points in the train set: 196651, number of used features: 287
[LightGBM] [Info] Start training from score 5124.313550
RMSE LightGBM: 1699.60334261121


### XGBoost

In [12]:
# Entrenar el modelo
xgb_model = xgb.XGBRegressor(random_state=42, n_jobs=-1)
xgb_model.fit(X_train, y_train)

# Hacer predicciones
y_pred_xgb = xgb_model.predict(X_test)

# Calcular el RMSE
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
print(f'RMSE XGBoost: {rmse_xgb}')



RMSE XGBoost: 1662.2433400763052


### CatBoost

In [13]:
# Crear un directorio temporal para CatBoost
with tempfile.TemporaryDirectory() as tmpdirname:
    print(f'Creando directorio temporal para CatBoost: {tmpdirname}')
    # Entrenar el modelo
    cat_model = CatBoostRegressor(random_state=42, train_dir=tmpdirname, verbose=0)
    cat_model.fit(X_train, y_train)


    # Hacer predicciones
    y_pred_cat = cat_model.predict(X_test)

    # Calcular el RMSE
    rmse_cat = np.sqrt(mean_squared_error(y_test, y_pred_cat))
    print(f'RMSE CatBoost: {rmse_cat}')


Creando directorio temporal para CatBoost: C:\Users\admin\AppData\Local\Temp\tmp_iqso8tu
RMSE CatBoost: 1608.7782028013598


### Tiempo de ejecucion

In [14]:
# Función para medir el tiempo de entrenamiento y predicción
def measure_time(model, X_train, y_train, X_test):
    start_train = time.time()
    
    model.fit(X_train, y_train)
    
    end_train = time.time()
    
    training_time = end_train - start_train

    start_predict = time.time()
    
    y_pred = model.predict(X_test)
    
    end_predict = time.time()
    
    prediction_time = end_predict - start_predict
    
    return training_time, prediction_time

# Medir los tiempos
models = {
    "Linear Regression": lr,
    "Decision Tree": dt,
    "Random Forest": rf,
    "LightGBM": lgb_model,
    "XGBoost": xgb_model,
    "CatBoost": cat_model
}

# Crear la tabla de resultados
results = []
for name, model in models.items():
    train_time, predict_time = measure_time(model, X_train, y_train, X_test)
    results.append([name, f'{train_time:.4f}s', f'{predict_time:.4f}s'])
    
# Imprimir la tabla
print(tabulate(results, headers=["Model", "Training Time", "Prediction Time"], tablefmt="grid"))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002740 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1176
[LightGBM] [Info] Number of data points in the train set: 196651, number of used features: 287
[LightGBM] [Info] Start training from score 5124.313550
+-------------------+-----------------+-------------------+
| Model             | Training Time   | Prediction Time   |
+===================+=================+===================+
| Linear Regression | 0.8253s         | 0.0360s           |
+-------------------+-----------------+-------------------+
| Decision Tree     | 3.4176s         | 0.0566s           |
+-------------------+-----------------+-------------------+
| Random Forest     | 20.3963s        | 0.2571s           |
+-------------------+-----------------+-------------------+
| LightGBM          | 0.5866s         | 0.1085s  

Modelos más rápidos de entrenar:

El modelo más rápido de entrenar fue LightGBM con un tiempo de entrenamiento de 0.5957 segundos.
Le siguen XGBoost (0.8463 segundos) y Linear Regression (0.8335 segundos).


Modelos más rápidos de predicción:<br>
CatBoost fue el modelo más rápido en hacer predicciones con un tiempo de predicción de solo 0.0130 segundos.
Después, se encuentra Linear Regression con 0.0340 segundos y XGBoost con 0.0654 segundos.



Modelo más lento de entrenar:<br>
Random Forest fue el modelo más lento de entrenar con un tiempo de 20.3850 segundos. Esto se debe a su naturaleza de ensamblaje que implica la construcción de múltiples árboles.


Modelo más lento de predicción:<br>
Random Forest también fue el modelo más lento en hacer predicciones con 0.2030 segundos. Aunque es rápido en entrenamiento comparado con otros modelos de ensamblaje, las predicciones son más costosas computacionalmente debido a la evaluación de múltiples árboles.


Performance general:<br>
LightGBM y XGBoost mostraron buenos tiempos tanto en entrenamiento como en predicción, destacándose por su eficiencia general.
CatBoost fue excepcionalmente rápido en predicción, lo que lo hace ideal para aplicaciones donde la velocidad de inferencia es crítica.